In [94]:
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
import torch
from torchmetrics import Perplexity
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from pathlib import Path
import pandas as pd


Initiate Configs

In [95]:
csv_path = Path("covid_articles_raw_first_250_.csv").absolute()
number_of_samples = 10
max_len = 1024
df = pd.read_csv(csv_path)[:number_of_samples]
# train_dataset = df.sample(frac=0.8, random_state=0)
# print(f"Loaded {len(train_dataset)} rows of training data")
# val_dataset = df.drop(train_dataset.index).sample(frac=0.5, random_state=0)
# print(f"Loaded {len(val_dataset)} rows of validation data")
# test_dataset = df.drop(train_dataset.index).drop(val_dataset.index)
# print(f"Loaded {len(test_dataset)} rows of test data")


Dataloader

In [96]:
class BartDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame):
        self.dataframe = dataframe
    def __getitem__(self, idx):
        return {
            'input_ids': self.dataframe['input_ids'].to_list()[idx],
            'attention_mask': self.dataframe['attention_mask'].to_list()[idx],
            'labels': self.dataframe['input_ids'].to_list()[idx]
        }
    def __len__(self):
        return len(self.dataframe['input_ids'])  # len(self.labels)


In [97]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn", max_len = max_len, truncation = True, padding = True)
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn", max_position_embeddings = max_len,ignore_mismatched_sizes=True)


encodings = tokenizer(df["content"].tolist(), max_length=max_len, truncation=True, padding=True)
decodings = tokenizer(df["title"].tolist(), max_length=max_len, truncation=True, padding=True)
full_df = pd.DataFrame({'input_ids': encodings['input_ids'], 'attention_mask': encodings['attention_mask'], 'labels': decodings['input_ids']})
print(full_df.head())
train_df = full_df.sample(frac=0.8, random_state=0)
train_dataset = BartDataset(train_df)
# val = summarizationDataLoader(val_dataset,tokenizer=tokenizer,max_len=max_len)
# test = summarizationDataLoader(test_dataset,tokenizer=tokenizer,max_len=max_len)

print(train_dataset[0])





Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /Users/alexiskaldany/.cache/huggingface/hub/models--facebook--bart-large-cnn/snapshots/c5121e42f57eca153aea31729f71cbedcd77a656/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-large-cnn",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_b

                                           input_ids  \
0  [0, 30086, 6, 99, 32, 47, 546, 13, 116, 50118,...   
1  [0, 4030, 469, 431, 10, 638, 1814, 6, 24083, 9...   
2  [0, 30086, 6, 99, 32, 47, 546, 13, 116, 50118,...   
3  [0, 30086, 6, 99, 32, 47, 546, 13, 116, 50118,...   
4  [0, 30086, 6, 99, 32, 47, 546, 13, 116, 50118,...   

                                      attention_mask  \
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
2  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
4  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   

                                              labels  
0  [0, 17629, 88, 5, 3157, 59, 2297, 7637, 11534,...  
1  [0, 725, 3463, 139, 4885, 4755, 613, 1997, 2, ...  
2  [0, 21318, 10369, 6, 2769, 7102, 11, 18900, 59...  
3  [0, 36017, 18, 809, 26484, 9923, 81, 33640, 11...  
4  [0, 574, 3209, 38230, 9118, 394, 5340, 27157, ..

In [98]:
seq2seqtraining_args = Seq2SeqTrainingArguments(
    output_dir="./summarization",
    num_train_epochs=2,
)

seq2seq_trainer = Seq2SeqTrainer(
    model=model,
    args=seq2seqtraining_args,
    train_dataset=train,
    eval_dataset=val,
    tokenizer=tokenizer,
)

seq2seq_trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/Users/alexiskaldany/school/nlp22_final/.venv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2


  0%|          | 0/2 [00:00<?, ?it/s]

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


AttributeError: 'list' object has no attribute 'keys'